In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
import json
import gzip
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from wordcloud import WordCloud

# Run these commands in your terminal or notebook before execution:
#
# pip install pandas numpy matplotlib scikit-learn plotly wordcloud pyarrow fastparquet nbformat

## Introduction:

1. Identify the predictive task you will study.
○ Describe how you will evaluate your model at this predictive task
○ What relevant baselines can be used for comparison
○ How you will assess the validity of your model’s predictions?

## Data collection and Pre-processing:

In [15]:
items = pd.read_parquet("items.parquet")
reviews = pd.read_parquet("reviews.parquet")

The full dataset includes the following:
* Reviews: 7,793,069
* Users: 2,567,538
* Items: 15,474
* Bundles: 615

And also metadata for genres, prices, discount history, and bundle composition.

The following table is used for variable descriptions and information for the items dataset:

| Variable Name   | Type           | Description                                                                                                                                                                                                                                                                                        |
| --------------- | -------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **user_id**     | string         | The username and Steam profile identifier. Can be customized (ex: "js41637") or numeric SteamID64.                                                                                                                                                                                                    |
| **items_count** | integer        | Total number of Steam items (games, DLCs, software) owned by the user.                                                                                                                                                                                                                             |
| **steam_id**    | string/integer | The numeric SteamID64 corresponding to the user; used to join with other Steam metadata.                                                                                                                                                                                                           |
| **user_url**    | string         | Direct link to the user’s Steam profile.                                                                                                                                                                                                                                                           |
| **items**       | list of dicts  | A list where each entry contains metadata about a game the user owns. Each dictionary typically includes: <br>• **item_id** (string): Unique game ID on Steam <br>• **item_name** (string): Name of the game <br>• (Optionally) **genre**, **discounted_price**, etc. depending on dataset version |


In [16]:
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


The following table is used for variable descriptions and information for the reviews dataset:

| Variable Name | Type          | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| ------------- | ------------- | ---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **user_id**   | string        | The ID of the user posting the reviews. Matches `user_id` in the items dataset.                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| **user_url**  | string        | Direct link to the user’s Steam profile; same as in items dataset.                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| **reviews**   | list of dicts | A list of all reviews the user has written. Each review dictionary includes fields like: <br>• **funny**: Number of "funny" reactions or empty string if none <br>• **helpful**: Rating such as `"15 of 20 people found this helpful"` or `"No ratings yet"` <br>• **id**: Steam review ID <br>• **item_id**: Game being reviewed <br>• **posted**: Date/time the review was posted <br>• **recommend**: Boolean indicating if the user recommended the game <br>• **review_text**: The free-text written review |


In [17]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'helpful': '15 of 20 people (75..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'helpful': 'No ratings yet', 'i..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'helpful': '2 of 2 people (100%..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [18]:
items['items'].iloc[0]

array([{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_2weeks': 0, 'playtime_forever': 6},
       {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_2weeks': 0, 'playtime_forever': 7},
       {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '60', 'item_name': 'Ricochet', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '70', 'item_name': 'Half-Life', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_2weeks': 0, 'playtime_forever': 4733},
       {'item_id': '240', 'item_name': 'Counter-Strik

In [19]:
reviews['reviews'].iloc[17]

array([{'funny': '17 people found this review funny', 'helpful': '28 of 31 people (90%) found this review helpful', 'item_id': '220200', 'last_edited': '', 'posted': 'Posted February 17, 2015.', 'recommend': True, 'review': "I'm going to keep this somewhat short and sweetAt the time of writing, Kerbal space program is in beta and there is talk of how soon it will be until it emerges from beta. But since its release in July 2011, it grown to become an outstanding example of Enthusiasm, Humor, and Profound triamph in a game. No matter whether your accomplishing great feats of exploration and scientific achievement......Or maybe you watched your beloved ship erupt into flames and debris becuase of a fatally unseen flaw. There is always something spectacular to be seen (Like your blown-up ship) or something to have a laugh at (Again like your blown-up ship)My recommendation?Buy this game, do something great, do something ridiculous, mod it, find something else just as great and just as cra

### Raw Data Information:
When loaded, the dataset contains deeply nested lists, which cannot be analyzed directly. The schema is as follows:
Each user has a list of all their owned games (items)
Each user has another list of all their written reviews (reviews)
Review texts vary in length and formatting
Some reviews are empty
Some entries have missing fields

To work with individual reviews, the nested structure was flattened:

In [20]:
def flatten_reviews(df):
    """Expand the nested review arrays into a flat DataFrame."""
    rows = []
    for user_reviews in df['reviews']:
        if isinstance(user_reviews, (list, np.ndarray)):
            for r in user_reviews:
                if isinstance(r, dict) and 'review' in r:
                    rows.append({
                        'user_id': r.get('user_id', None),
                        'item_id': r.get('item_id', None),
                        'review': r.get('review', '').strip(),
                        'recommend': r.get('recommend', False)
                    })
    return pd.DataFrame(rows)

flat_reviews = flatten_reviews(reviews)

# Drop missing or empty reviews
flat_reviews = flat_reviews[flat_reviews['review'].str.len() > 5].reset_index(drop=True)

print(f"Flattened reviews: {len(flat_reviews)} rows")
print(flat_reviews.head(3))

Flattened reviews: 57407 rows
  user_id item_id                                             review  \
0    None    1250  Simple yet with great replayability. In my opi...   
1    None   22200               It's unique and worth a playthrough.   
2    None   43110  Great atmosphere. The gunplay can be a bit chu...   

   recommend  
0       True  
1       True  
2       True  


To remove lower valued entries that could be considered spam, reviews shorter than 5 characters were removed.
We also computed review length for our future model.

In [21]:
flat_reviews = flat_reviews[flat_reviews['review'].str.len() > 5].reset_index(drop=True)
flat_reviews['review_length'] = flat_reviews['review'].str.split().apply(len)

In [22]:
summary_table = pd.DataFrame({
    "Total Reviews": [len(flat_reviews)],
    "Positive (Recommend=True)": [flat_reviews['recommend'].sum()],
    "Negative (Recommend=False)": [(~flat_reviews['recommend']).sum()],
    "Average Review Length": [flat_reviews['review_length'].mean()],
    "Median Review Length": [flat_reviews['review_length'].median()]
})
print(summary_table)


   Total Reviews  Positive (Recommend=True)  Negative (Recommend=False)  \
0          57407                      50738                        6669   

   Average Review Length  Median Review Length  
0               39.83819                  15.0  


To see the overall distribution of the data we currently have, we explored the structure of review text and recommendation behavior:
Review lengths are highly skewed: Most reviews are short (median = 15 words), but the mean is much higher (≈ 40 words), indicating some extremely long reviews.
Positive reviews dominate (88% recommended), which is consistent with review platform positivity bias. 

https://www.sciencedirect.com/science/article/pii/S2667096823000319

This means ratings tend to be systematically higher than the actual average customer experience. Review platform positivity bias might be due to factors such as:
* **Self-Selection**: Consumers on the two extremes, positive or negative, are more likely to leave a review, while those with moderate opinions do not feel the need or effort to write one.
* **Acquisition Selection Bias**: Reviews are from purchasers who were already positively predisposed toward the product or service (ex. liking the franchise) leading to a more positive pool of feedback.
* **Social Influence Bias**: New reviewers may be influenced by existing positive ratings and rate their experience more positively to conform.

The fact that negative reviews tend to be longer might be an indicator that players write longer explanations when they dislike a game, or have more to critique.

By flattening millions of reviews into a row-level format, we created a dataset suitable for:

* Sentiment analysis
* Supervised classification
* Text-based feature engineering
* recommendation modeling

This transforms our raw nested metadata into a structured format ready for machine learning models.

## Exploratory Analysis:

In [23]:
fig1 = px.histogram(flat_reviews, x='recommend', color='recommend',
                    title="Distribution of Recommendations",
                    labels={'recommend': 'Recommend'},
                    color_discrete_map={True: 'green', False: 'red'})
fig1.show()

# Review Length Distribution
fig2 = px.histogram(flat_reviews, x='review_length', nbins=50, color='recommend',
                    title="Distribution of Review Lengths by Recommendation",
                    labels={'review_length': 'Word Count'},
                    color_discrete_map={True: 'green', False: 'red'})
fig2.show()

# Boxplot of Review Lengths
fig3 = px.box(flat_reviews, x='recommend', y='review_length', color='recommend',
              title="Boxplot of Review Lengths by Recommendation",
              labels={'recommend': 'Recommend', 'review_length': 'Word Count'},
              color_discrete_map={True: 'green', False: 'red'})
fig3.show()

# Correlation Gauge
corr_val = flat_reviews[['review_length', 'recommend']].corr().iloc[0, 1]
fig4 = go.Figure(go.Indicator(
    mode="gauge+number",
    value=corr_val,
    title={'text': "Correlation: Review Length vs Recommendation"},
    gauge={'axis': {'range': [-1, 1]}, 'bar': {'color': "blue"}}
))
fig4.show()

# --- Step 5: Word Clouds (Matplotlib) ---
positive_text = " ".join(flat_reviews[flat_reviews['recommend']]['review'].tolist())
negative_text = " ".join(flat_reviews[~flat_reviews['recommend']]['review'].tolist())

pos_wc = WordCloud(width=800, height=400, background_color='white').generate(positive_text)
neg_wc = WordCloud(width=800, height=400, background_color='black', colormap='Reds').generate(negative_text)

plt.figure(figsize=(10, 5))
plt.imshow(pos_wc, interpolation='bilinear')
plt.axis("off")
plt.title("Positive Reviews Word Cloud")
plt.show()

plt.figure(figsize=(10, 5))
plt.imshow(neg_wc, interpolation='bilinear')
plt.axis("off")
plt.title("Negative Reviews Word Cloud")
plt.show()

NameError: name 'WordCloud' is not defined

In [ ]:
tfidf = TfidfVectorizer(
    stop_words='english',
    max_features=5000,
    ngram_range=(1,2)
)

X = tfidf.fit_transform(flat_reviews['review'])
y = flat_reviews['recommend'].astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)

dt_param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2']
}

dt_grid = GridSearchCV(
    estimator=dt,
    param_grid=dt_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

dt_grid.fit(X_train, y_train)

print("\n🟣 Best Decision Tree Parameters:", dt_grid.best_params_)
print("🟣 Best Decision Tree CV Score:", dt_grid.best_score_)

# Create final DT model with best params
best_dt = dt_grid.best_estimator_



🟣 Best Decision Tree Parameters: {'criterion': 'gini', 'max_depth': 20, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 10}
🟣 Best Decision Tree CV Score: 0.8914534567229179


In [ ]:
# DECISION TREE EVALUATION
y_pred_dt = best_dt.predict(X_test)
bal_acc_dt = balanced_accuracy_score(y_test, y_pred_dt) 

print("\n=== Decision Tree Evaluation ===")
print(f"Balanced Accuracy: {bal_acc_dt:.4f}")
print(classification_report(y_test, y_pred_dt, digits=3))

cm_dt = confusion_matrix(y_test, y_pred_dt)
print("\nConfusion Matrix:")
print(cm_dt)



=== Decision Tree Evaluation ===
Balanced Accuracy: 0.5909
              precision    recall  f1-score   support

           0      0.629     0.197     0.300      1334
           1      0.903     0.985     0.942     10148

    accuracy                          0.893     11482
   macro avg      0.766     0.591     0.621     11482
weighted avg      0.871     0.893     0.868     11482


Confusion Matrix:
[[ 263 1071]
 [ 155 9993]]


In [ ]:
rf = RandomForestClassifier(random_state=42)

rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'max_features': ['sqrt', 'log2'],
}

rf_grid = GridSearchCV(
    estimator=rf,
    param_grid=rf_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

rf_grid.fit(X_train, y_train)

print("🔵 Best Random Forest Parameters:", rf_grid.best_params_)
print("🔵 Best Random Forest CV Score:", rf_grid.best_score_)

# Create final RF model with best params
best_rf = rf_grid.best_estimator_

🔵 Best Random Forest Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
🔵 Best Random Forest CV Score: 0.898704409363092


In [ ]:
# RANDOM FOREST EVALUATION
y_pred_rf = best_rf.predict(X_test)
bal_acc_rf = balanced_accuracy_score(y_test, y_pred_rf)

print("\n=== Random Forest Evaluation ===")
print(f"Balanced Accuracy: {bal_acc_rf:.4f}")
print(classification_report(y_test, y_pred_rf, digits=3))

cm_rf = confusion_matrix(y_test, y_pred_rf)
print("\nConfusion Matrix:")
print(cm_rf)



=== Random Forest Evaluation ===
Balanced Accuracy: 0.6082
              precision    recall  f1-score   support

           0      0.667     0.232     0.344      1334
           1      0.907     0.985     0.944     10148

    accuracy                          0.897     11482
   macro avg      0.787     0.608     0.644     11482
weighted avg      0.879     0.897     0.875     11482


Confusion Matrix:
[[ 309 1025]
 [ 154 9994]]


In [ ]:
logreg = LogisticRegression(max_iter=2000)

logreg_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear'],
    'class_weight': [None, 'balanced'],
    'max_iter': [500, 1000, 2000],
    'tol': [1e-4, 1e-3, 1e-2]
}


logreg_grid = GridSearchCV(
    estimator=logreg,
    param_grid=logreg_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

logreg_grid.fit(X_train, y_train)

print("🟠 Best Logistic Regression Params:", logreg_grid.best_params_)
print("🟠 Best Logistic Regression CV Score:", logreg_grid.best_score_)

best_logreg = logreg_grid.best_estimator_


🟠 Best Logistic Regression Params: {'C': 1, 'class_weight': None, 'max_iter': 500, 'penalty': 'l1', 'solver': 'liblinear', 'tol': 0.01}
🟠 Best Logistic Regression CV Score: 0.9098530212302668


In [ ]:
# LOGISTIC REGRESSION EVALUATION
y_pred_logreg = best_logreg.predict(X_test)
bal_acc_logreg = balanced_accuracy_score(y_test, y_pred_logreg)

print("\n=== Logistic Regression Evaluation ===")
print(f"Balanced Accuracy: {bal_acc_logreg:.4f}")
print(classification_report(y_test, y_pred_logreg, digits=3))

cm_logreg = confusion_matrix(y_test, y_pred_logreg)
print("\nConfusion Matrix:")
print(cm_logreg)


=== Logistic Regression Evaluation ===
Balanced Accuracy: 0.6719
              precision    recall  f1-score   support

           0      0.739     0.361     0.485      1334
           1      0.921     0.983     0.951     10148

    accuracy                          0.911     11482
   macro avg      0.830     0.672     0.718     11482
weighted avg      0.900     0.911     0.897     11482


Confusion Matrix:
[[ 481  853]
 [ 170 9978]]


In [ ]:
xgb = XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=42
)

xgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'min_child_weight': [1, 3],
    'subsample': [0.7, 1.0],
}

xgb_grid = GridSearchCV(
    estimator=xgb,
    param_grid=xgb_param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

xgb_grid.fit(X_train, y_train)

print("🟡 Best XGBoost Params:", xgb_grid.best_params_)
print("🟡 Best XGBoost CV Score:", xgb_grid.best_score_)

best_xgb = xgb_grid.best_estimator_


Fitting 5 folds for each of 32 candidates, totalling 160 fits


c:\Users\Kevin\miniforge3\envs\dsc80\Lib\site-packages\xgboost\core.py:158: UserWarning:

[22:49:21] WARNING: D:\bld\xgboost-split_1737531313485\work\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




🟡 Best XGBoost Params: {'learning_rate': 0.1, 'max_depth': 5, 'min_child_weight': 3, 'n_estimators': 200, 'subsample': 0.7}
🟡 Best XGBoost CV Score: 0.9031899836690258


In [ ]:
# XGBOOST EVALUATION
y_pred_xgb = best_xgb.predict(X_test)
bal_acc_xgb = balanced_accuracy_score(y_test, y_pred_xgb)

print("\n=== XGBoost Evaluation ===")
print(f"Balanced Accuracy: {bal_acc_xgb:.4f}")
print(classification_report(y_test, y_pred_xgb, digits=3))

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
print("\nConfusion Matrix:")
print(cm_xgb)



=== XGBoost Evaluation ===
Balanced Accuracy: 0.6023
              precision    recall  f1-score   support

           0      0.772     0.213     0.334      1334
           1      0.906     0.992     0.947     10148

    accuracy                          0.901     11482
   macro avg      0.839     0.602     0.640     11482
weighted avg      0.890     0.901     0.875     11482


Confusion Matrix:
[[  284  1050]
 [   84 10064]]


In [ ]:
def show_top_features(model, vectorizer, n=15):
    feature_names = np.array(vectorizer.get_feature_names_out())
    coefs = model.coef_[0]
    top_pos = np.argsort(coefs)[-n:]
    top_neg = np.argsort(coefs)[:n]
    print("\nTop Positive (Recommend=True):")
    print(feature_names[top_pos])
    print("\nTop Negative (Recommend=False):")
    print(feature_names[top_neg])

show_top_features(best_logreg, tfidf)


Top Positive (Recommend=True):
['awsome' 'bom' 'epic' '10 10' 'addicting' 'highly' 'fun' 'great'
 'fantastic' 'definitely' 'love' 'best' 'excellent' 'amazing' 'awesome']

Top Negative (Recommend=False):
['worst' 'unplayable' 'horrible' 'poorly' 'boring' 'terrible' 'repetitive'
 'overkill' 'uninstalled' 'disappointing' 'refund' 'microtransactions'
 'garbage' 'uninstall' 'awful']


1. Identify the predictive task you will study
## Task Definition
Predict whether a Steam user will **recommend (`True`)** or **not recommend (`False`)** a game based solely on the text of their written review.  
This is a **binary classification task** using supervised learning.

---

## Model Type
**Logistic Regression classifier** trained on **TF-IDF text features**.

---

## 📏 Evaluation Metrics

**Primary:**
- Balanced Accuracy  
- F1-Score *(handles class imbalance)*  

**Secondary:**
- Precision  
- Recall  
- ROC-AUC  

---

## Baselines for Comparison
- Majority-class predictor *(“always recommend”)*  
- Random guessing baseline  
- Simple keyword heuristic *(e.g., review contains “recommend”)*  

---

## Validity of Predictions
- **Confusion matrix** → to inspect Type I / Type II errors  
- **Feature weight analysis** → identify top positive and negative words  
- **Cross-validation** → to verify model robustness and stability  



2. Exploratory analysis, data collection, pre-processing, and discussion:

3. Modeling:

4. Evaluation:

5. Discussion of related work:

In [ ]:
# ## potential alternative dataset to use

# import ast

# path = "steam_reviews.json.gz"

# rows = []
# with gzip.open(path, 'rt', encoding='utf-8') as f:
#     for line in f:
#         line = line.strip()
#         if not line:
#             continue
#         try:
#             rows.append(ast.literal_eval(line))
#         except Exception as e:
#             # Skip any malformed lines
#             print("Skipping bad line:", e)
#             continue

# df = pd.DataFrame(rows)
# print(df.shape)
# print(df.columns)
# df.head()